In [169]:
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
import multiprocessing as mp
import numpy as np
import pandas as pd
import sys
import time
from scipy.stats import entropy
from math import log, e
from scipy.stats.stats import pearsonr  


from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from skopt import BayesSearchCV

plt.rc('axes', titlesize=20) 
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=20)

## Load the data

In [2]:
data_raw = pd.read_csv('AP221Harvardx.csv')

/Users/dvamvou/anaconda3/envs/ds/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (4,27,28,45,46,47,48,49,55,56,59,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data_raw.head(5)

,course_id,user_id,registered,viewed,explored,certified,completed,ip,cc_by_ip,countryLabel,...,roles_isInstructor,roles_isStaff,roles_isCCX,roles_isFinance,roles_isLibrary,roles_isSales,forumRoles_isAdmin,forumRoles_isCommunityTA,forumRoles_isModerator,forumRoles_isStudent
0,HarvardX/PH525.1x/1T2018,814784,True,True,False,False,False,74.12.190.49,CA,Canada,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,HarvardX/PH525.1x/1T2018,7945078,True,False,NaN,False,False,60.10.17.72,CN,China,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,HarvardX/PH525.1x/1T2018,5465534,True,True,False,False,False,109.196.85.139,PL,Poland,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,HarvardX/PH525.1x/1T2018,8087152,True,True,False,False,False,203.110.242.10,IN,India,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,HarvardX/PH525.1x/1T2018,5032030,True,False,NaN,False,False,96.82.226.121,US,United States,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [4]:
data_raw.shape

(6860993, 91)

## Create a sub-dataset with some of the quasi-identifiers plus the grade

In [5]:
quasi_identifiers = 'course_id YoB countryLabel LoE gender nforum_posts'.split()
non_qi_fields = ['grade']
df = data_raw[quasi_identifiers + non_qi_fields]

In [6]:
df.shape

(6860993, 7)

In [7]:
df.head()

,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade
0,HarvardX/PH525.1x/1T2018,NaN,Canada,NaN,NaN,NaN,NaN
1,HarvardX/PH525.1x/1T2018,NaN,China,NaN,NaN,NaN,NaN
2,HarvardX/PH525.1x/1T2018,NaN,Poland,NaN,NaN,NaN,NaN
3,HarvardX/PH525.1x/1T2018,NaN,India,NaN,NaN,NaN,NaN
4,HarvardX/PH525.1x/1T2018,NaN,United States,NaN,NaN,NaN,NaN


In [2]:
filename = 'partial_dataset.csv'

In [9]:
df.to_csv(filename, index=False)

In [3]:
df_original = pd.read_csv(filename)

In [4]:
df_original.head()

,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade
0,HarvardX/PH525.1x/1T2018,NaN,Canada,NaN,NaN,NaN,NaN
1,HarvardX/PH525.1x/1T2018,NaN,China,NaN,NaN,NaN,NaN
2,HarvardX/PH525.1x/1T2018,NaN,Poland,NaN,NaN,NaN,NaN
3,HarvardX/PH525.1x/1T2018,NaN,India,NaN,NaN,NaN,NaN
4,HarvardX/PH525.1x/1T2018,NaN,United States,NaN,NaN,NaN,NaN


In [5]:
df_original.isna().sum()

course_id             0
YoB             1049890
countryLabel    1109868
LoE              998390
gender           914661
nforum_posts    6623522
grade           1930278
dtype: int64

In [165]:
df_original['countryLabel'].value_counts(normalize=True).shape

(254,)

In [164]:
df_original['countryLabel'].value_counts(normalize=True)

United States                     3.215562e-01
India                             8.051659e-02
United Kingdom                    4.258541e-02
Brazil                            3.990889e-02
Canada                            3.853490e-02
China                             2.717399e-02
Australia                         2.169854e-02
Germany                           1.940664e-02
Mexico                            1.914512e-02
Spain                             1.716203e-02
France                            1.580039e-02
Colombia                          1.241183e-02
Netherlands                       1.128040e-02
Russian Federation                1.075685e-02
Turkey                            1.074868e-02
Pakistan                          9.995262e-03
Egypt                             9.821209e-03
Greece                            9.424069e-03
Philippines                       9.339042e-03
Japan                             9.081354e-03
Nigeria                           9.007107e-03
Singapore    

## Data cleaning - set nforum_posts which are NaN to 0

In [6]:
df_original_cleaned = df_original.copy(deep=True)

In [7]:
df_original_cleaned.loc[df_original_cleaned['nforum_posts'].isna(), 'nforum_posts'] = 0

In [8]:
df_original_cleaned.isna().sum()

course_id             0
YoB             1049890
countryLabel    1109868
LoE              998390
gender           914661
nforum_posts          0
grade           1930278
dtype: int64

In [9]:
df_original_cleaned.describe()

,YoB,nforum_posts,grade
count,5.811103e+06,6.860993e+06,4.930715e+06
mean,1.984321e+03,2.458084e-01,4.063551e-02
std,1.234031e+01,3.767435e+00,1.656013e-01
min,5.130000e+02,0.000000e+00,-5.000000e-02
25%,1.979000e+03,0.000000e+00,0.000000e+00
50%,1.988000e+03,0.000000e+00,0.000000e+00
75%,1.993000e+03,0.000000e+00,0.000000e+00
max,3.116000e+03,3.770000e+03,1.200000e+00


Calculate the original correlation between `nforum_posts` and `grade`.

In [103]:
df_original_corr = df_original_cleaned['nforum_posts'].corr(df_original_cleaned['grade'], method='pearson')
df_original_corr

0.2699651746572297

## Load data with synthetic rows 

In [11]:
synthetic_df = pd.read_csv('synthetic_dataset_supersets.csv')

In [12]:
synthetic_df.head()

,Unnamed: 0,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade
0,0,HarvardX/PH525.1x/1T2018,NaN,Canada,NaN,NaN,0.0,NaN
1,1,HarvardX/PH525.1x/1T2018,NaN,China,NaN,NaN,0.0,NaN
2,2,HarvardX/PH525.1x/1T2018,NaN,Poland,NaN,NaN,0.0,NaN
3,3,HarvardX/PH525.1x/1T2018,NaN,India,NaN,NaN,0.0,NaN
4,4,HarvardX/PH525.1x/1T2018,NaN,United States,NaN,NaN,0.0,NaN


In [83]:
synthetic_df.shape

(12180246, 8)

In [84]:
synthetic_df.isna().sum()

Unnamed: 0            0
course_id             0
YoB             1087585
countryLabel    1146438
LoE             1077012
gender           952979
nforum_posts          0
grade           1930278
dtype: int64

In [97]:
grade_marginal_distribution = synthetic_df[(synthetic_df['grade'].notnull()) & (synthetic_df['grade'] != 9999.0)]['grade'].values

In [207]:
df_original.loc[df_original['grade'].notnull()].shape

(4930715, 7)

In [208]:
synthetic_df.loc[(synthetic_df['grade']!=9999)&(synthetic_df['grade'].notnull())].shape

(4930715, 12)

## Sample grade from the marginal distribution

In [101]:
samples_from_marginal_grade_dist = np.random.choice(grade_marginal_distribution, size = synthetic_df[synthetic_df['grade']==9999].shape[0], \
                 replace = True)
synthetic_df['grade_impute_marginal'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['grade_impute_marginal']==9999.0, 'grade_impute_marginal'] = samples_from_marginal_grade_dist

In [113]:
corr_bias = df_original_corr - synthetic_df['grade_impute_marginal'].corr(synthetic_df['nforum_posts'], method='pearson')

In [114]:
marginal_mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['grade_impute_marginal'].mean()
print(f'Bias in mean grade for imputation with random sample from marginal is {marginal_mean_bias}')
print(f'Bias in correlation between grade and nforum_posts for imputation with random sample from marginal is {corr_bias}')


Bias in mean grade for imputation with random sample from marginal is 2.300478505520842e-05
Bias in correlation between grade and nforum_posts for imputation with random sample from marginal is 0.18596957483374393


## Average multiple samples from the marginal distribution to get grade

In [117]:
def average_samples_from_marginal(x):
    samples = np.random.choice(grade_marginal_distribution, size = 5, replace = True)
    return samples.mean()
                        
                   

In [120]:
synthetic_df['grade_impute_marginal_mean'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['grade_impute_marginal_mean']==9999.0, 'grade_impute_marginal_mean'] = \
synthetic_df.loc[synthetic_df['grade_impute_marginal_mean']==9999.0, 'grade_impute_marginal_mean'].apply(average_samples_from_marginal)

In [122]:
corr_bias = df_original_corr - synthetic_df['grade_impute_marginal_mean'].corr(synthetic_df['nforum_posts'], method='pearson')
marginal_mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['grade_impute_marginal_mean'].mean()
print(f'Bias in mean grade for imputation with random sample from marginal is {marginal_mean_bias}')
print(f'Bias in correlation between grade and nforum_posts for imputation with random sample from marginal is {corr_bias}')



Bias in mean grade for imputation with random sample from marginal is -1.0785173819326255e-05
Bias in correlation between grade and nforum_posts for imputation with random sample from marginal is 0.16001774574717303


## Tree-based algorithms to fill in the missing values of the grades

In [33]:
#code to encode labels for multiple categorical predictors

from sklearn.preprocessing import LabelEncoder

class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [78]:
def calc_meanstd(X_train, y_train, depths, folds, scoring):
    cvmeans = []
    train_scores = []
    cvstds = []
    for depth in depths:
        dt = tree.DecisionTreeRegressor(max_depth=depth)
        cv_scores = cross_val_score(dt, X_train, y_train, cv=folds, scoring=scoring)
        cvmeans.append(cv_scores.mean())
        cvstds.append(cv_scores.std())
        dt.fit(X_train, y_train)
        train_scores.append(dt.score(X_train, y_train))
    cvmeans = np.array(cvmeans)
    cvstds = np.array(cvstds)
    return cvmeans, cvstds


## How to handle missing values?

For categorical predictors, we encode `NaN` as another category. For `YoB`, we filled any missing values with the median year of birth. We also changed the `countryLabel` column to a binary one to encode whether the country is US (1) or not (0), thus reducing the number of possible categories that the column can take.

In [59]:
df_filled = df_original_cleaned.copy(deep=True)
df_filled[['countryLabel', 'LoE', 'gender']] = df_filled[['countryLabel', 'LoE', 'gender']].fillna('Null')

In [60]:
df_filled['countryLabel_bin'] = 0
df_filled.loc[df_filled['countryLabel']=='United States', 'countryLabel_bin'] = 1
df_filled.head()

,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade,countryLabel_bin
0,HarvardX/PH525.1x/1T2018,NaN,Canada,Null,Null,0.0,NaN,0
1,HarvardX/PH525.1x/1T2018,NaN,China,Null,Null,0.0,NaN,0
2,HarvardX/PH525.1x/1T2018,NaN,Poland,Null,Null,0.0,NaN,0
3,HarvardX/PH525.1x/1T2018,NaN,India,Null,Null,0.0,NaN,0
4,HarvardX/PH525.1x/1T2018,NaN,United States,Null,Null,0.0,NaN,1


In [63]:
df_filled['YoB'] = df_filled['YoB'].fillna(df_filled['YoB'].median())
df_filled.head()

,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade,countryLabel_bin
0,HarvardX/PH525.1x/1T2018,1988.0,Canada,Null,Null,0.0,NaN,0
1,HarvardX/PH525.1x/1T2018,1988.0,China,Null,Null,0.0,NaN,0
2,HarvardX/PH525.1x/1T2018,1988.0,Poland,Null,Null,0.0,NaN,0
3,HarvardX/PH525.1x/1T2018,1988.0,India,Null,Null,0.0,NaN,0
4,HarvardX/PH525.1x/1T2018,1988.0,United States,Null,Null,0.0,NaN,1


In [97]:
multi_label_enc = MultiColumnLabelEncoder(columns = ['course_id','LoE','gender'])

In [98]:
df_filled_encoded = multi_label_enc.fit_transform(df_filled)
df_filled_encoded.head()

,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade,countryLabel_bin
0,166,1988.0,Canada,1,0,0.0,NaN,0
1,166,1988.0,China,1,0,0.0,NaN,0
2,166,1988.0,Poland,1,0,0.0,NaN,0
3,166,1988.0,India,1,0,0.0,NaN,0
4,166,1988.0,United States,1,0,0.0,NaN,1


In [216]:
df_encoded_grade_not_null = df_filled_encoded.loc[df_filled_encoded['grade'].notnull()]
X = df_encoded_grade_not_null[['course_id', 'YoB', 'LoE', 'gender', 'nforum_posts','countryLabel_bin']].values
y = df_encoded_grade_not_null['grade'].values

In [217]:
print(X.shape)
print(y.shape)

(4930715, 6)
(4930715,)


In [79]:
depths = np.arange(10,100,10)
cvmeans, cvstds = calc_meanstd(X, y, depths, folds=3, scoring='r2')

In [83]:
cvmeans

array([-0.70619219, -2.6001578 , -2.827799  , -2.82755096, -2.82567621,
       -2.82292562, -2.83235772, -2.83166247, -2.82155569])

In [84]:
depths = np.arange(10,40,5)
cvmeans_rmse, cvstds_rmse = calc_meanstd(X, y, depths, folds=3, scoring='neg_mean_squared_error')

In [85]:
cvmeans_rmse

array([-0.06075985, -0.10365384, -0.11447625, -0.11962043, -0.1200914 ,
       -0.12019464])

In [246]:
dt_10 = tree.DecisionTreeRegressor(max_depth=10)
dt_10.fit(X, y)

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=None, splitter='best')

## Imputation with Decision Tree predicted grade

In [247]:
synthetic_df.loc[synthetic_df['grade']==9999.0]

,Unnamed: 0,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade,grade_dt10_pred,grade_rf_pred,adaboost_pred,ada_tuned_pred
6860993,0,HarvardX/ER22.1x/2015T3,1977.0,Haiti,hs,f,0.0,9999.0,0.020885,0.032812,0.143697,0.031448
6860994,1,HarvardX/ER22.1x/2015T3,1977.0,Haiti,hs,f,0.0,9999.0,0.034760,0.032812,0.081410,0.007819
6860995,2,HarvardX/ER22.1x/2015T3,1977.0,Haiti,hs,f,0.0,9999.0,0.043970,0.032812,0.106080,-0.001448
6860996,3,HarvardX/ER22.1x/2015T3,1977.0,Haiti,hs,f,0.0,9999.0,0.076576,0.032812,0.100350,0.048923
6860997,4,HarvardX/CS50x3/2015,1996.0,China,m,m,0.0,9999.0,-0.011029,0.000004,0.052303,0.015334
6860998,5,HarvardX/CS50x3/2015,1996.0,China,m,m,0.0,9999.0,-0.028877,0.000004,-0.013980,-0.008241
6860999,6,HarvardX/CS50x3/2015,1996.0,China,m,m,0.0,9999.0,-0.007106,0.000004,-0.008892,-0.021074
6861000,7,HarvardX/CS50x3/2015,1996.0,China,m,m,0.0,9999.0,0.010213,0.000004,-0.018521,-0.032555
6861001,8,HarvardX/1368.4x/2T2015,1997.0,Pakistan,hs,f,0.0,9999.0,0.065591,0.035069,0.152182,0.071836
6861002,9,HarvardX/1368.4x/2T2015,1997.0,Pakistan,hs,f,0.0,9999.0,0.054299,0.035069,0.207974,0.007770


In [99]:
test_set = synthetic_df.loc[synthetic_df['grade']==9999.0]
test_set[['countryLabel', 'LoE', 'gender']] = test_set[['countryLabel', 'LoE', 'gender']].fillna('Null')
test_set['countryLabel_bin'] = 0
test_set.loc[test_set['countryLabel']=='United States', 'countryLabel_bin'] = 1
test_set['YoB'] = test_set['YoB'].fillna(df_filled['YoB'].median())
test_set = multi_label_enc.transform(test_set)
test_set.head()

/Users/dvamvou/anaconda3/envs/ds/lib/python3.7/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
/Users/dvamvou/anaconda3/envs/ds/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/dvamvou/anaconda3/envs/ds/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,Unnamed: 0,course_id,YoB,countryLabel,LoE,gender,nforum_posts,grade,countryLabel_bin
6860993,0,45,1977.0,Haiti,5,1,0.0,9999.0,0
6860994,1,45,1977.0,Haiti,5,1,0.0,9999.0,0
6860995,2,45,1977.0,Haiti,5,1,0.0,9999.0,0
6860996,3,45,1977.0,Haiti,5,1,0.0,9999.0,0
6860997,4,37,1996.0,China,8,2,0.0,9999.0,0


In [101]:
X_test = test_set[['course_id', 'YoB', 'LoE', 'gender', 'nforum_posts','countryLabel_bin']].values


In [248]:
dt10_preds = dt_10.predict(X_test)

In [249]:
noise = np.random.normal(0, 0.025, size=len(dt10_preds))
synthetic_df['grade_dt10_pred'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['grade_dt10_pred']==9999.0, 'grade_dt10_pred'] = dt10_preds + noise

In [250]:
corr_bias = df_original_corr - synthetic_df['grade_dt10_pred'].corr(synthetic_df['nforum_posts'], method='pearson')
mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['grade_dt10_pred'].mean()
print(f'Bias in mean grade for imputation with predicted value from decision tree is {np.abs(mean_bias)}')
print(f'Bias in correlation between grade and nforum_posts for imputation with predicted value from decision tree is {np.abs(corr_bias)}')



Bias in mean grade for imputation with predicted value from decision tree is 0.03872050770831121
Bias in correlation between grade and nforum_posts for imputation with predicted value from decision tree is 0.05783441034780906


## Imputation with Random Forest predicted grade

In [223]:
rf = ensemble.RandomForestRegressor(n_estimators = 50, max_depth = 10, max_features = 5)
rf.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features=5, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [255]:
rf_preds = rf.predict(X_test)
noise = np.random.normal(0, 0.025, size=len(rf_preds))


synthetic_df['grade_rf_pred'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['grade_rf_pred']==9999.0, 'grade_rf_pred'] = rf_preds + noise

In [256]:
corr_bias = df_original_corr - synthetic_df['grade_rf_pred'].corr(synthetic_df['nforum_posts'], method='pearson')
mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['grade_rf_pred'].mean()
print(f'Bias in mean grade for imputation with predicted value from random forest is {np.abs(mean_bias)}')
print(f'Bias in correlation between grade and nforum_posts for imputation with predicted value from random forest is {np.abs(corr_bias)}')



Bias in mean grade for imputation with predicted value from random forest is 0.03436038547425271
Bias in correlation between grade and nforum_posts for imputation with predicted value from random forest is 0.08591764273627661


## Imputation with Bagging predicted grade

In [252]:
bagging = ensemble.RandomForestRegressor(n_estimators = 50, max_depth = 10, max_features = X.shape[1])
bagging.fit(X, y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features=6, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=50,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [260]:
bagging_preds = bagging.predict(X_test)
noise = np.random.normal(0, 0.025, size=len(bagging_preds))

synthetic_df['grade_bagging_pred'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['grade_bagging_pred']==9999.0, 'grade_bagging_pred'] = bagging_preds + noise

In [261]:
corr_bias = df_original_corr - synthetic_df['grade_bagging_pred'].corr(synthetic_df['nforum_posts'], method='pearson')
mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['grade_bagging_pred'].mean()
print(f'Bias in mean grade for imputation with predicted value from bagging is {np.abs(mean_bias)}')
print(f'Bias in correlation between grade and nforum_posts for imputation with predicted value from bagging is {np.abs(corr_bias)}')


Bias in mean grade for imputation with predicted value from bagging is 0.038374982646014945
Bias in correlation between grade and nforum_posts for imputation with predicted value from bagging is 0.06213311574046215


## Imputation with AdaBoost predicted grade

In [257]:
adaboost = ensemble.AdaBoostRegressor(n_estimators = 500, learning_rate = 0.1)
adaboost.fit(X, y)

AdaBoostRegressor(base_estimator=None, learning_rate=0.1, loss='linear',
                  n_estimators=500, random_state=None)

In [258]:
adaboost_preds = adaboost.predict(X_test)
noise = np.random.normal(0, 0.025, size=len(adaboost_preds))

synthetic_df['adaboost_pred'] = synthetic_df['grade']
synthetic_df.loc[synthetic_df['adaboost_pred']==9999.0, 'adaboost_pred'] = adaboost_preds + noise

In [259]:
corr_bias = df_original_corr - synthetic_df['adaboost_pred'].corr(synthetic_df['nforum_posts'], method='pearson')
mean_bias = df_original_cleaned['grade'].mean() - synthetic_df['adaboost_pred'].mean()
print(f'Bias in mean grade for imputation with predicted value from adaboost is {np.abs(mean_bias)}')
print(f'Bias in correlation between grade and nforum_posts for imputation with predicted value from adaboost is {np.abs(corr_bias)}')


Bias in mean grade for imputation with predicted value from adaboost is 0.07210078917198387
Bias in correlation between grade and nforum_posts for imputation with predicted value from adaboost is 0.05477595511579658
